In [23]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_excel('../../data/data_ford_price.xlsx') 

In [6]:
data[data.isna().any(axis=1)]

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN
8,18750,2011,2,6,77000,clean,automatic,rwd,NaN,33.698900,-84.745400,59.0
12,1700,2001,1,8,275000,clean,automatic,4wd,NaN,44.549400,-90.611200,45.0
18,32500,2015,3,6,73300,clean,automatic,4wd,full-size,49.184885,-122.801111,NaN
21,32700,2015,3,6,88500,clean,automatic,4wd,full-size,50.039078,-125.271885,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6991,6600,2010,2,8,101215,clean,automatic,rwd,NaN,43.098946,-76.097853,53.0
7007,28500,2014,3,8,40000,clean,automatic,4wd,NaN,36.562900,-82.653200,59.0
7008,14995,2010,3,8,0,clean,automatic,NaN,full-size,32.788557,-117.098278,59.0
7011,30750,2016,4,6,66000,clean,automatic,4wd,NaN,37.132840,-95.785580,51.0


### Задание 4.2
Какая доля строк в датасете останется, если убрать пропуски в столбце size? Ответ округлите до двух знаков после точки-разделителя.

In [14]:
display(round(1-data['size'].isnull().mean(),2))
np.round(len(data[~data['size'].isna()])/len(data),2)

0.78

In [15]:
# a=data[data.isna().any(axis=1)].shape[0]
# a=data[~data['size'].isna()].shape[0]
# b=data.shape[0]
# print(round(1/b*a,2))

In [24]:
def encode_cat_features(columns_to_change, X_train, X_test, y_train):
    one_hot_encoder = OneHotEncoder()
    # учим и создаём новые признаки
    X_train_onehot = one_hot_encoder.fit_transform(X_train[columns_to_change]).toarray()
    # создаём новые признаки
    X_test_onehot = one_hot_encoder.transform(X_test[columns_to_change]).toarray()
    # получаем список с названием новых признаков
    columns = one_hot_encoder.get_feature_names_out(columns_to_change)
    
    # преобразуем массив в DataFrame с названием колонок новых признаков
    X_train_onehot_df = pd.DataFrame(X_train_onehot, columns=columns)
    X_test_onehot_df = pd.DataFrame(X_test_onehot, columns=columns)

    # удаляем колонку с index
    X_train = X_train.reset_index().drop(['index'], axis = 1)
    X_test = X_test.reset_index().drop(['index'], axis = 1)
    y_train = y_train.reset_index().drop(['index'], axis = 1)

    # объединяем DataFrame с новыми признаками и старые данные
    X_train_new = pd.concat([X_train, X_train_onehot_df], axis=1)
    X_test_new = pd.concat([X_test, X_test_onehot_df], axis=1)
    # в общем DataFrame удаляем колонки с признаками которые нужно было кодировать
    X_train_new = X_train_new.drop(columns=columns_to_change)
    X_test_new = X_test_new.drop(columns=columns_to_change)

    return X_train_new, X_test_new

In [16]:
y = data['price']
x = data.drop(columns='price')

In [44]:
data = x.copy()

# записали всё с пропусками 
test_data = data[data['weather'].isnull()]
# удалили все пропуски
data.dropna(inplace=True)

# это наша цель для обучения без пропусков 
y_train = data['weather']
# данные для обучения получены с данных без пропусков
X_train = data.drop(['size','weather','drive'], axis=1)
# данные для предсказания получены с данных с пропуском
X_test = test_data.drop(['size','weather','drive'], axis=1)


In [45]:
categorial_cols = ['cylinders', 'title_status', 'transmission']

In [46]:
# запуск кодирования
X_train_new, X_test_new = encode_cat_features(categorial_cols, X_train, X_test, y_train)

In [47]:
model = LinearRegression()
model.fit(X_train_new, y_train)
 
y_pred = model.predict(X_test_new)
y_pred

array([ 40.91435555,  40.7637233 ,  39.74866152,  41.2755305 ,
        40.31791932,  41.10796547,  41.15337846,  39.94866488,
        41.10796547,  40.7217165 ,  40.18904454,  91.62094167,
        41.12549856,  41.33052316,  39.66827354,  40.91435555,
        40.77287826,  40.84208674,  41.10796547,  41.02118034,
        40.31791932,  41.30309209,  40.77645269,  40.75842615,
        40.61605044,  40.79031628,  40.7701239 ,  39.78723017,
        41.27231621,  39.77492057,  40.7637233 ,  40.7701239 ,
        41.10796547,  39.68313064,  40.12277414,  39.7873657 ,
        41.07798631,  41.06812063,  40.7637233 ,  40.90194049,
        41.10796547,  70.85737739,  30.44339508,  40.76125291,
        39.77492057,  41.12549856,  39.74864816,  40.72219488,
        40.97162064,  40.7448585 ,  39.71065847,  39.78779447,
        39.77492057,  40.15945849,  41.12549856,  40.7902779 ,
        39.65555168,  41.10796547,  40.76125291,  40.15619215,
        40.79031628,  41.10796547,  40.7902779 ,  40.72

### Задание 4.5 (на самопроверку)
Вставьте найденную замену на место пропусков в столбце weather. Используйте тот же метод для заполнения пропусков в size. Обратите внимание на то, какой тип модели нужен (классификация или регрессия) в зависимости от типа признака.

Попробуйте выполнить задание самостоятельно. При затруднениях вы можете посмотреть решение в ноутбуке в начале юнита.

In [77]:
# list_a=list(x[x['weather'].isna()]['weather'].index)
# b=len(y_pred)
# if len(list_a)==b:
#     for i in range(b):
#         x.loc[list_a[i],'weather']=y_pred[i]

In [78]:
for i, ni in enumerate(test_data.index):
    x.loc[ni, 'weather'] = y_pred[i]